In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import json
from glob import glob
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
import time
import multiprocessing
from tqdm import tqdm
from pathlib import Path
import pickle
from datetime import datetime, timedelta
from scipy.interpolate import make_interp_spline, BSpline
import covidcast
import geonamescache
import time
from datetime import datetime, date

import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

matplotlib.rcParams.update({
    "pgf.preamble": [
         "\\usepackage{arev}",
        "\\usepackage[T1]{fontenc}"]
})

pd.set_option('display.max_columns', None)
time_retrieval = datetime.now()

time_produced = datetime.now()

# hospitalization

In [ ]:
"""
hospital admissions data from CMU
API documentation here: https://cmu-delphi.github.io/delphi-epidata/api/covidcast_signals.html
hospitalization documentation: https://cmu-delphi.github.io/delphi-epidata/api/covidcast-signals/hospital-admissions.html
"""
start = time.time()

data = covidcast.signal(data_source="hospital-admissions",
                        signal="smoothed_adj_covid19",
                        start_day=date(2020, 1, 1),
                        end_day=date(2020, 11, 30),
                        geo_type="state")


print('time taken to get data:', time.time() - start, 'secs. shape:', data.shape)


data['geo_value'] = data['geo_value'].astype(str)

hospital_admissions_df = data

print('% states we have data for:', len(pd.unique(hospital_admissions_df['geo_value']))/51*100, pd.unique(hospital_admissions_df['geo_value']) )


hospital_admissions_trimmed_df = hospital_admissions_df[[
    "geo_value", #FIPS
    "time_value", #date
    "value" #hospitalization number
]]

hospital_admissions_trimmed_df.columns = ["state", "day", "hospitalization"]

hospital_admissions_trimmed_df['day'] = pd.to_datetime(hospital_admissions_trimmed_df['day'])
print(np.min(hospital_admissions_trimmed_df['day']), np.max(hospital_admissions_trimmed_df['day']))
print(hospital_admissions_trimmed_df.dtypes)
hospital_admissions_trimmed_df.head()

# deaths

In [ ]:
"""
doctor's visits data from CMU
API documentation here: https://cmu-delphi.github.io/delphi-epidata/api/covidcast_signals.html
signal documentation: https://cmu-delphi.github.io/delphi-epidata/api/covidcast-signals/indicator-combination.html
"""
start = time.time()
data = covidcast.signal(data_source="jhu-csse",
                        signal="deaths_7dav_incidence_prop",
                        start_day=date(2020, 1, 1),
                        end_day=date(2020, 11, 30),
                        geo_type="state")


data['geo_value'] = data['geo_value'].astype(str)

deaths_df = data

print('before dropping NA', deaths_df.shape)
deaths_df = deaths_df[deaths_df['geo_value'].notna()]
print('after dropping NA', deaths_df.shape)

print('% states we have data for:', len(pd.unique(deaths_df['geo_value']))/51*100, pd.unique(deaths_df['geo_value']) )


deaths_trimmed_df = deaths_df[[
    "geo_value", #FIPS
    "time_value", #date
    "value" #hospitalization number
]]

deaths_trimmed_df.columns = ["state", "day", "deaths"]

deaths_trimmed_df['day'] = pd.to_datetime(deaths_trimmed_df['day'])
print(np.min(deaths_trimmed_df['day']), np.max(deaths_trimmed_df['day']))
print(deaths_trimmed_df.dtypes)
deaths_trimmed_df.head()

# confirmed cases

In [ ]:
"""
doctor's visits data from CMU
API documentation here: https://cmu-delphi.github.io/delphi-epidata/api/covidcast_signals.html
signal documentation: https://cmu-delphi.github.io/delphi-epidata/api/covidcast-signals/indicator-combination.html
"""
start = time.time()

data = covidcast.signal(data_source="jhu-csse",
                        signal="confirmed_7dav_incidence_prop",
                        start_day=date(2020, 1, 1),
                        end_day=date(2020, 11, 30),
                        geo_type="state")



data = pd.read_csv('data/input/confirmed_cases_{}.csv'.format(time_retrieval))

data['geo_value'] = data['geo_value'].astype(str)

confirmed_cases_df = data

print('before dropping NA', confirmed_cases_df.shape)
confirmed_cases_df = confirmed_cases_df[confirmed_cases_df['geo_value'].notna()]
print('after dropping NA', confirmed_cases_df.shape)

print('% states we have data for:', len(pd.unique(confirmed_cases_df['geo_value']))/51*100, pd.unique(confirmed_cases_df['geo_value']) )


confirmed_cases_trimmed_df = confirmed_cases_df[[
    "geo_value", #FIPS
    "time_value", #date
    "value" #hospitalization number
]]

confirmed_cases_trimmed_df.columns = ["state", "day", "confirmed_cases"]

confirmed_cases_trimmed_df['day'] = pd.to_datetime(confirmed_cases_trimmed_df['day'])
print(np.min(confirmed_cases_trimmed_df['day']), np.max(confirmed_cases_trimmed_df['day']))

print(confirmed_cases_trimmed_df.dtypes)
confirmed_cases_trimmed_df.head()

# merging all outcome variables

In [ ]:
from functools import reduce
merged_outcome_df = reduce(lambda x,y: pd.merge(x , y, left_on=['state', 'day'], right_on=['state', 'day'] , how='inner'), 
                           [hospital_admissions_trimmed_df,
#                             doctor_visits_trimmed_df,
                            confirmed_cases_trimmed_df,
                            deaths_trimmed_df
                           ])


merged_outcome_df.head()

# google mobility data

In [ ]:
"""
# merging with mobility data 
# https://www.google.com/covid19/mobility/
"""

data = pd.read_csv('data/input/Global_Mobility_Report.csv.zip')
google_mobility_agg_agg = data[data['country_region'] == "United States"]
google_mobility_agg_agg['date'] = pd.to_datetime(google_mobility_agg_agg['date'])
google_mobility_agg_agg = google_mobility_agg_agg[['sub_region_1', 'date', 'retail_and_recreation_percent_change_from_baseline', 'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline']] 
google_mobility_agg_agg = google_mobility_agg_agg.groupby(['sub_region_1', 'date']).agg(['mean'])
google_mobility_agg_agg.reset_index(inplace=True)


google_mobility_agg_agg.columns = [
'sub_region_1', 'date', 'retail_and_recreation', 
    'grocery_and_pharmacy',
       'parks',
       'transit_stations',
       'workplaces',
       'residential']

for column in ['grocery_and_pharmacy',
'retail_and_recreation',
       'parks',
       'transit_stations',
       'workplaces',
       'residential']:
    
    google_mobility_agg_agg[column] = (google_mobility_agg_agg[column]+100)/100
    google_mobility_agg_agg[column+'_sq'] = google_mobility_agg_agg[column]**2


for column in ['grocery_and_pharmacy',
'retail_and_recreation',
       'parks',
       'transit_stations',
       'workplaces',
       'residential']:
    print('plotting ', column)
    plot_data = google_mobility_agg_agg
    fig, ax = plt.subplots(figsize=(8, 5))
    for label, grp in plot_data.groupby('sub_region_1'):
        x = grp['date']
        y = grp[column]
        plt.plot(x, y, label = label, alpha = 0.8)
    plt.title(column)
#     plt.legend(loc="lower left", ncol=16)
    plt.ylabel(column, fontsize=16)


print(pd.unique(google_mobility_agg_agg['sub_region_1']))    
print(np.min(google_mobility_agg_agg['date']), np.max(google_mobility_agg_agg['date']))
    
google_mobility_agg_agg.head()

# covid testing data

In [ ]:
# data from: 
# https://covidtracking.com/data/api 
# https://covidtracking.com/about-data/data-definitions

testing_data = pd.read_csv('data/input/all-states-history_dec01.csv')
testing_data = testing_data[['date', 'state', 'totalTestResultsIncrease']]
testing_data['date'] = pd.to_datetime(testing_data['date'])
testing_data.head()

# getting population to normalize testing rate by population

In [ ]:
gc = geonamescache.GeonamesCache()
states_dict = gc.get_us_states()
states_df = pd.DataFrame()
# transforming it into a pandas dataframe for merge later
for key in states_dict.keys():
    statecode = key
#     print(key)
    states_df = states_df.append({
    "statecode": statecode,
    "Statename": states_dict[statecode]['name']
    }, ignore_index=True)

# del counties_list
# print(states_df.head())

population_df = pd.read_csv('data/input/state_population.csv', thousands=',')
# print(population_df.head())

merged_population = population_df.merge(states_df,
                                       left_on = ['State'],
                                         right_on = ['Statename'],
                                         how = 'left')

print('population_df', population_df.shape,
      '\n states_df data', states_df.shape,
      '\nmerge', merged_population.shape)

testing_data_pop_df = testing_data.merge(merged_population,
                                         left_on = ['state'],
                                         right_on = ['statecode'],
                                         how = 'left')   

print('testing_data', testing_data.shape,
      '\n merged_population', merged_population.shape,
      '\nmerge', testing_data_pop_df.shape)

testing_data_pop_df['new_test_rate'] = testing_data_pop_df['totalTestResultsIncrease']/testing_data_pop_df['population']
# print(testing_data_pop_df.head())

testing_data_pop_df = testing_data_pop_df[['date', 'state', 'Statename', 'new_test_rate', 'totalTestResultsIncrease', 'population']]

testing_data_pop_df.head()

# merged_population
# testing_data_pop_df

In [ ]:
np.min(testing_data_pop_df['date']), np.max(testing_data_pop_df['date'])

In [ ]:
merged_outcome_df['state'] = merged_outcome_df['state'].str.upper()

merged_outcome_with_control = merged_outcome_df.merge(testing_data_pop_df, 
                                         left_on = ['state', 'day'],
                                         right_on = ['state', 'date'])


merged_outcome_with_control = merged_outcome_with_control[['state', 'date', 'Statename', 'hospitalization', 
#                                                            'hospitalization_abs', 'beds_100K','hospitalization_100K','inpatient_beds_used',                                                           
                                                           'confirmed_cases', 
                                                           'deaths', 'new_test_rate',
                                                           'totalTestResultsIncrease']]
merged_outcome_with_control.head()

In [ ]:

merged_outcome_with_control = merged_outcome_with_control.merge(google_mobility_agg_agg, 
                                         left_on = ['Statename', 'date'],
                                         right_on = ['sub_region_1', 'date'])

merged_outcome_with_control = merged_outcome_with_control[['state', 'date', 'Statename', 'hospitalization', 
                                                           'confirmed_cases',
#                                                            'beds_100K', 'inpatient_beds_used','hospitalization_abs', 'hospitalization_100K',
                                                           'deaths', 'new_test_rate',
                                                           'totalTestResultsIncrease', 'retail_and_recreation',
                                                           'grocery_and_pharmacy', 'parks', 'transit_stations', 'workplaces',
                                                           'residential', 'grocery_and_pharmacy_sq', 'retail_and_recreation_sq',
                                                           'parks_sq', 'transit_stations_sq', 'workplaces_sq', 'residential_sq']]

merged_outcome_with_control.head()

# getting temperature and humidity data

In [ ]:
# this data is prepared by 1a-weather_data_us_states.ipynb

NOAA_precipitation = pd.read_csv('data/input/NOAA_precipitation.csv')
NOAA_precipitation['date'] = pd.to_datetime(NOAA_precipitation['date'])
NOAA_temperature = pd.read_csv('data/input/NOAA_temperature.csv')
NOAA_temperature['date'] = pd.to_datetime(NOAA_temperature['date'])

print(NOAA_precipitation.head())
print(NOAA_temperature.head())

In [ ]:
merged_outcome_with_control_test = merged_outcome_with_control.merge(NOAA_precipitation,
                                                  left_on = ['Statename', 'date'],                                                  
                                                  right_on = ['location_name', 'date'],
                                                how = 'left', # since we don't have weather data for AK, HI and DC
#                                                  indicator=True 
                                                  )

print('NOAA_precipitation', NOAA_precipitation.shape,
      '\n merged_outcome_with_control', merged_outcome_with_control.shape,
      '\nmerge', merged_outcome_with_control_test.shape)

merged_outcome_with_control_test = merged_outcome_with_control_test.merge(NOAA_temperature,
                                                  left_on = ['Statename', 'date'],                                                  
                                                  right_on = ['location_name', 'date'],
                                                how = 'left', 
#                                                  indicator=True 
                                                  )

print('NOAA_temperature', NOAA_temperature.shape,
      '\n merged_outcome_with_control', merged_outcome_with_control.shape,
      '\nmerge', merged_outcome_with_control_test.shape)

merged_outcome_with_control = merged_outcome_with_control_test[['state', 'date', 'Statename', 'hospitalization', 
#                                                                 'hospitalization_abs', 'hospitalization_100K',
#                                                                 'beds_100K', 'inpatient_beds_used',
       'confirmed_cases', 'deaths', 'new_test_rate',
       'totalTestResultsIncrease',
       'retail_and_recreation',
       'grocery_and_pharmacy', 'parks', 'transit_stations', 'workplaces',
       'residential', 'grocery_and_pharmacy_sq', 'retail_and_recreation_sq',
       'parks_sq', 'transit_stations_sq', 'workplaces_sq', 'residential_sq',
       'precipitation_avg', 'temperature_avg']]


merged_outcome_with_control.head()

In [ ]:
np.min(merged_outcome_with_control_test['date']), np.max(merged_outcome_with_control_test['date'])

# reading mandate data

In [ ]:
# data from https://github.com/USCOVIDpolicy/COVID-19-US-State-Policy-Database/blob/master/COVID-19%20US%20state%20policy%20database%2011_9_2020.xlsx
# https://docs.google.com/spreadsheets/d/1zu9qEWI8PsOI_i8nI_S29HDGHlIp2lfVMsGxpQ5tvAQ/edit#gid=1894978869

NPI_data = pd.read_excel('data/input/COVID-19 US state policy database 11_30_2020 corrected.xlsx',
                        sheet_name = 'State policy changes ',
#                       dtype = {"POSTCODE": str, "FM_ALL": str, "FMFINE": str, "FMCITE": str, "FMNOENF": str, "FM_EMP": str, "FM_END": str, "FM_STP": str},
                        usecols = ["POSTCODE", 
                                   "FM_ALL", "FM_EMP", "FM_END", #mask stuff
                                  "STAYHOME", "STAYHOMENOGP", "END_STHM", #stay at home
                                   "CLBSNS", "END_BSNS" # Closed other non-essential businesses
                                  ],
                        skiprows = [1,2,3]
                        
                        )
NPI_data = NPI_data.tail(53) # remove 'unit' row
NPI_data = NPI_data.head(51) # remove trailing empty rows
# NPI_data.head()
NPI_data

In [ ]:
# putting states with no mask mandates as a huge date in the future (where samurais wake up) for assignment to a dummy later
NPI_data['FM_ALL'] = NPI_data['FM_ALL'].replace({0: "2077-12-31"}) 
NPI_data['FM_ALL'] = pd.to_datetime(NPI_data['FM_ALL'])

NPI_data['FM_EMP'] = NPI_data['FM_EMP'].replace({0: "2077-12-31"})
NPI_data['FM_EMP'] = pd.to_datetime(NPI_data['FM_EMP'])

NPI_data['FM_END'] = NPI_data['FM_END'].replace({0: "2077-12-31"})
NPI_data['FM_END'] = pd.to_datetime(NPI_data['FM_END'])

NPI_data['STAYHOME'] = NPI_data['STAYHOME'].replace({0: "2077-12-31"})
NPI_data['STAYHOME'] = pd.to_datetime(NPI_data['STAYHOME'])

NPI_data['STAYHOMENOGP'] = NPI_data['STAYHOMENOGP'].replace({0: "2077-12-31"})
NPI_data['STAYHOMENOGP'] = pd.to_datetime(NPI_data['STAYHOMENOGP'])

NPI_data['END_STHM'] = NPI_data['END_STHM'].replace({0: "2077-12-31"})
NPI_data['END_STHM'] = pd.to_datetime(NPI_data['END_STHM'])

NPI_data['CLBSNS'] = NPI_data['CLBSNS'].replace({0: "2077-12-31"})
NPI_data['CLBSNS'] = pd.to_datetime(NPI_data['CLBSNS'])

NPI_data['END_BSNS'] = NPI_data['END_BSNS'].replace({0: "2077-12-31"})
NPI_data['END_BSNS'] = pd.to_datetime(NPI_data['END_BSNS'])

NPI_data.head()

In [ ]:
NPI_data['earliest_policy_start'] = NPI_data[['FM_ALL','FM_EMP']].min(axis=1)
NPI_data['earliest_STAYHOME_start'] = NPI_data[['STAYHOME','STAYHOMENOGP']].min(axis=1)

NPI_data_trimmed = NPI_data[['POSTCODE', 'FM_END', 'earliest_policy_start', 
                             "earliest_STAYHOME_start", "END_STHM",
                             "CLBSNS", "END_BSNS"]]
print(NPI_data.dtypes)
NPI_data_trimmed.head()

# merging outcome with mandates

In [ ]:
outcome_with_mandate = merged_outcome_with_control.merge(NPI_data_trimmed, 
                                         left_on = ['state'],
                                         right_on = ['POSTCODE'],
                                         how='inner')
print(merged_outcome_with_control.shape,
    NPI_data_trimmed.shape,
    outcome_with_mandate.shape)

outcome_with_mandate.head()

In [ ]:
# masks
outcome_with_mandate['delta_end'] = outcome_with_mandate['date'] - outcome_with_mandate['FM_END']
outcome_with_mandate['delta_end'] = outcome_with_mandate['delta_end'].astype('timedelta64[D]').astype(int)  
outcome_with_mandate['delta_day'] = outcome_with_mandate['date'] - outcome_with_mandate['earliest_policy_start']
outcome_with_mandate['delta_day'] = outcome_with_mandate['delta_day'].astype('timedelta64[D]').astype(int)  

#stay at home
outcome_with_mandate['STAYHOME_delta_end'] = outcome_with_mandate['date'] - outcome_with_mandate['END_STHM']
outcome_with_mandate['STAYHOME_delta_end'] = outcome_with_mandate['STAYHOME_delta_end'].astype('timedelta64[D]').astype(int)  
outcome_with_mandate['STAYHOME_delta_day'] = outcome_with_mandate['date'] - outcome_with_mandate['earliest_STAYHOME_start']
outcome_with_mandate['STAYHOME_delta_day'] = outcome_with_mandate['STAYHOME_delta_day'].astype('timedelta64[D]').astype(int)  

#stay at home
outcome_with_mandate['business_delta_end'] = outcome_with_mandate['date'] - outcome_with_mandate['CLBSNS']
outcome_with_mandate['business_delta_end'] = outcome_with_mandate['business_delta_end'].astype('timedelta64[D]').astype(int)  
outcome_with_mandate['business_delta_day'] = outcome_with_mandate['date'] - outcome_with_mandate['CLBSNS']
outcome_with_mandate['business_delta_day'] = outcome_with_mandate['business_delta_day'].astype('timedelta64[D]').astype(int)  

outcome_with_mandate.head()

In [ ]:
outcome_with_mandate.loc[outcome_with_mandate['delta_day'] < -1000, 'delta_day'] = -10000
outcome_with_mandate.loc[outcome_with_mandate['STAYHOME_delta_day'] < -1000, 'STAYHOME_delta_day'] = -10000
outcome_with_mandate.loc[outcome_with_mandate['business_delta_day'] < -1000, 'business_delta_day'] = -10000

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
# plt.hist( outcome_with_mandate.loc[outcome_with_mandate['delta_day'] > -1000, 'delta_day'], bins = 30 )
plt.hist( outcome_with_mandate['delta_day'], bins = 30 )
plt.xticks(rotation=90)

In [ ]:
outcome_with_mandate['time'] = outcome_with_mandate['date'] - pd.to_datetime('2019-11-01')
outcome_with_mandate['time'] = outcome_with_mandate['time'].astype('timedelta64[D]').astype(int)  

outcome_with_mandate['time'].hist()

outcome_with_mandate.head()

# not taking data that's after a mask mandate ended

In [ ]:
weekly_w_mandates_df = outcome_with_mandate
weekly_w_mandates_df = weekly_w_mandates_df[
    (weekly_w_mandates_df['delta_end'] < 0)
#     & (weekly_w_mandates_df['STAYHOME_delta_end'] < 0) 
]

print(outcome_with_mandate.shape, weekly_w_mandates_df.shape)

# doing a z-normalization for outcome variables

In [ ]:
temp_normed = weekly_w_mandates_df[['state', 'confirmed_cases', 'deaths']]
temp_normed = temp_normed.groupby(['state']).agg(['mean', 'std'])
temp_normed.reset_index(inplace = True)
temp_normed.columns = ['state', 
                       'confirmed_cases_mean', 'confirmed_cases_std',
                      'deaths_mean', 'deaths_std',
#                        'hospitalization_100K_mean', 'hospitalization_100K_std',
#                        'hospitalization_abs_mean', 'hospitalization_abs_std',
                      ]

weekly_w_mandates_df = weekly_w_mandates_df.merge(temp_normed)

weekly_w_mandates_df['confirmed_cases_normed'] = (weekly_w_mandates_df['confirmed_cases']-weekly_w_mandates_df['confirmed_cases_mean'])/weekly_w_mandates_df['confirmed_cases_std']
weekly_w_mandates_df['deaths_normed'] = (weekly_w_mandates_df['deaths']-weekly_w_mandates_df['deaths_mean'])/weekly_w_mandates_df['deaths_std']
# weekly_w_mandates_df['hospitalization_abs_normed'] = (weekly_w_mandates_df['hospitalization_abs']-weekly_w_mandates_df['hospitalization_abs_mean'])/weekly_w_mandates_df['hospitalization_abs_std']
# weekly_w_mandates_df['hospitalization_100K_normed'] = (weekly_w_mandates_df['hospitalization_100K']-weekly_w_mandates_df['hospitalization_100K_mean'])/weekly_w_mandates_df['hospitalization_100K_std']

weekly_w_mandates_df.head()

# temp_normed.head()

# creating delayed values for control

In [ ]:
weekly_w_mandates_df['hospitalization_delay_14days'] = weekly_w_mandates_df['hospitalization'].shift(14)
weekly_w_mandates_df['hospitalization_delay_14days_growth'] = (weekly_w_mandates_df['hospitalization'] - weekly_w_mandates_df['hospitalization_delay_14days'])/weekly_w_mandates_df['hospitalization_delay_14days']


weekly_w_mandates_df['deaths_normed_delay_14days'] = weekly_w_mandates_df['deaths_normed'].shift(14)
weekly_w_mandates_df['deaths_normed_delay_14days_growth'] = (weekly_w_mandates_df['deaths_normed'] - weekly_w_mandates_df['deaths_normed_delay_14days'])/weekly_w_mandates_df['deaths_normed_delay_14days']


weekly_w_mandates_df['confirmed_cases_normed_delay_14days'] = weekly_w_mandates_df['confirmed_cases_normed'].shift(14)
weekly_w_mandates_df['confirmed_cases_normed_delay_14days_growth'] = (weekly_w_mandates_df['confirmed_cases_normed'] - weekly_w_mandates_df['confirmed_cases_normed_delay_14days'])/weekly_w_mandates_df['confirmed_cases_normed_delay_14days']


weekly_w_mandates_df.head(15)

# daily data export

In [ ]:
daily_data_df = weekly_w_mandates_df[weekly_w_mandates_df['delta_end'] < 0]

print(daily_data_df.shape)
daily_data_df.to_csv('data/output/daily_data_with_other_NPI_df_produced_{}.csv'.format(str(time_produced.strftime("%Y-%m-%d %H.%M"))), index=False)
print('data/output/daily_data_with_other_NPI_df_produced_{}.csv'.format(str(time_produced.strftime("%Y-%m-%d %H.%M"))))
daily_data_df.head()

# robustness

# trying to see if mandates have an effect on compliance for late states

In [ ]:
# dates from https://delphi.cmu.edu/blog/2020/10/12/new-and-improved-covid-symptom-survey-tracks-testing-and-mask-wearing/
# documentation: https://cmu-delphi.github.io/delphi-epidata/api/covidcast-signals/fb-survey.html

mask_adherance = covidcast.signal(data_source="fb-survey",
                        signal="smoothed_wwearing_mask",
                        start_day=date(2020, 9, 1),
                        end_day=date(2020, 12, 8),
                        geo_type="state")


print('time taken to get data:', time.time() - start, 'secs. shape:', data.shape)
mask_adherance.head()

In [ ]:
mask_adherance_agg = mask_adherance[mask_adherance['sample_size'] > 0]
print(mask_adherance_agg.shape, mask_adherance.shape)
mask_adherance_agg = mask_adherance_agg[['geo_value', 'time_value', 'value']]
mask_adherance_agg.columns = [
    'state', 'date', 'compliance'
]

mask_adherance_agg['date'] = pd.to_datetime(mask_adherance_agg['date'])

mask_adherance_agg.head()

In [ ]:

names = {
    1: 'Hawaii',
    2: 'Iowa',
    3: 'North Dakota',
    4: 'New Hampshire'
}

dates = {
    1: '2020-11-16 00:00',
    2: '2020-11-16 07:00',
    3: '2020-11-14',
    4: '2020-11-20'
}

color = {
    1: 'red',
    2: 'green',
    3: 'blue',
    4: 'orange'
}

fig, ax = plt.subplots(figsize=(12, 8))
for i, state in enumerate(['HI', 'IA', 'ND', 'NH']):    
    plot_data = mask_adherance_agg[mask_adherance_agg['state'] == state.lower()]
    plt.plot(plot_data['date'], 
             plot_data['compliance'],
             label = names[i+1],
             color = color[i+1]
        )
#     ax.axvline(pd.to_datetime(dates[i+1]), color = color[i+1])
    
    
plt.legend(loc="lower right", ncol=4)

plt.ylabel("Mask Adherence (%)", fontsize=16)
plt.xlabel("Date", fontsize=16)

    

In [ ]:
mask_adherance_select = mask_adherance_agg[mask_adherance_agg['state'].isin(['hi', 'ia', 'nd', 'nh'])]
mask_adherance_select = mask_adherance_select.sort_values(by=['date'], ascending = False)
mask_adherance_select.head()

In [ ]:
# mask_adherance_select = mask_adherance_agg[mask_adherance_agg['state'].isin(['HI', 'IA', 'ND', 'NH'])]
mask_adherance_select = mask_adherance_agg[mask_adherance_agg['state'].isin(['hi', 'ia', 'nd', 'nh'])]

print(mask_adherance_select.shape, mask_adherance_agg.shape)

late_state_mandates = pd.read_csv('data/input/late_state_mandates.csv')
late_state_mandates['mandate_startdate'] = pd.to_datetime(late_state_mandates['mandate_startdate'])

mask_adherance_merge = mask_adherance_select.merge(late_state_mandates,
                                                  right_on = 'state',
                                                  left_on = 'state',                                                  
                                                  )

print('mask_adherance_select', mask_adherance_select.shape,
      '\n late_state_mandates', late_state_mandates.shape,
      '\nmerge', mask_adherance_merge.shape)

mask_adherance_merge.head()

In [ ]:
mask_adherance_merge['delta_day'] = mask_adherance_merge['date'] - mask_adherance_merge['mandate_startdate']
mask_adherance_merge['delta_day'] = mask_adherance_merge['delta_day'].astype('timedelta64[D]').astype(int)  
mask_adherance_merge['delta_week'] = mask_adherance_merge['delta_day']/7.0
mask_adherance_merge['delta_week'] = mask_adherance_merge['delta_week'].astype(int)  

mask_adherance_merge.head()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
for state in ['HI', 'IA', 'ND', 'NH']:    
    plot_data = mask_adherance_merge[mask_adherance_merge['state'] == state.lower()]
    plt.plot(plot_data['delta_day'], 
             plot_data['compliance'],
             label = state
        )
plt.legend(loc="lower left", ncol=4)

In [ ]:
mask_adherance_merge = mask_adherance_merge.sort_values(by=['delta_day'], ascending = False)
mask_adherance_merge

In [ ]:
mask_adherance_merge_outcomes = mask_adherance_merge.merge(confirmed_cases_trimmed_df,
                                                  left_on = ['state', 'date'],                                                  
                                                  right_on = ['state', 'day'],
                                                  )
print('mask_adherance_merge', mask_adherance_merge.shape,
  
      '\nmerge', mask_adherance_merge_outcomes.shape)

mask_adherance_merge_outcomes = mask_adherance_merge_outcomes.merge(deaths_trimmed_df,
                                                  left_on = ['state', 'date'],                                                  
                                                  right_on = ['state', 'day'],
                                                  )

mask_adherance_merge_outcomes['state'] = mask_adherance_merge_outcomes['state'].str.upper()

print('mask_adherance_merge', mask_adherance_merge.shape,
      '\nmerge', mask_adherance_merge_outcomes.shape)

mask_adherance_merge_outcomes = mask_adherance_merge_outcomes.merge(testing_data_pop_df,
                                                  left_on = ['state', 'date'],                                                  
                                                  right_on = ['state', 'date'],
                                                  )

print('mask_adherance_merge', mask_adherance_merge.shape,
      '\nmerge', mask_adherance_merge_outcomes.shape)

mask_adherance_merge_outcomes = mask_adherance_merge_outcomes.merge(google_mobility_agg_agg,
                                                  left_on = ['Statename', 'date'],                                                  
                                                  right_on = ['sub_region_1', 'date'],
                                                  )



print('mask_adherance_merge', mask_adherance_merge.shape,
      '\nmerge', mask_adherance_merge_outcomes.shape)

mask_adherance_merge_outcomes.head()

In [ ]:
np.min(mask_adherance_merge_outcomes['date']), np.max(mask_adherance_merge_outcomes['date'])

In [ ]:
mask_adherance_merge_outcomes.to_csv('data/output/mask_adherance_merge_outcomes_produced_{}.csv'.format(str(time_produced.strftime("%Y-%m-%d %H.%M"))), 
                                     index=False)
print('data/output/mask_adherance_merge_outcomes_produced_{}.csv'.format(str(time_produced.strftime("%Y-%m-%d %H.%M"))))


# regressing covid outcomes on mask adherance

In [ ]:
mask_adherance_agg_temp = mask_adherance_agg
mask_adherance_agg_temp['state'] = mask_adherance_agg_temp['state'].str.lower()

mask_adherance_merge_outcomes_all_states = mask_adherance_agg_temp.merge(confirmed_cases_trimmed_df,
                                                  left_on = ['state', 'date'],                                                  
                                                  right_on = ['state', 'day'],
                                                  )
print(
    'mask_adherance_agg_temp', mask_adherance_agg_temp.shape,
      '\n merged_outcome_df', confirmed_cases_trimmed_df.shape,
#       '\n deaths_trimmed_df', deaths_trimmed_df.shape,      
#       '\n testing_data_pop_df', testing_data_pop_df.shape,
#       '\n google_mobility_agg_agg', google_mobility_agg_agg.shape,      
      '\nmerge', mask_adherance_merge_outcomes_all_states.shape)


mask_adherance_merge_outcomes_all_states = mask_adherance_merge_outcomes_all_states.merge(deaths_trimmed_df,
                                                  left_on = ['state', 'day'],                                                  
                                                  right_on = ['state', 'day'],
                                                  )



mask_adherance_merge_outcomes_all_states['state'] = mask_adherance_merge_outcomes_all_states['state'].str.upper()

print(
      '\nmerge', mask_adherance_merge_outcomes_all_states.shape)

mask_adherance_merge_outcomes_all_states = mask_adherance_merge_outcomes_all_states.merge(testing_data_pop_df,
                                                  left_on = ['state', 'date'],                                                  
                                                  right_on = ['state', 'date'],
                                                  )

print(
      '\nmerge', mask_adherance_merge_outcomes_all_states.shape)

mask_adherance_merge_outcomes_all_states = mask_adherance_merge_outcomes_all_states.merge(google_mobility_agg_agg,
                                                  left_on = ['Statename', 'date'],                                                  
                                                  right_on = ['sub_region_1', 'date'],
                                                  )



print(
      '\nmerge', mask_adherance_merge_outcomes_all_states.shape)


mask_adherance_merge_outcomes_all_states = mask_adherance_merge_outcomes_all_states.merge(NOAA_temperature,
                                                  left_on = ['Statename', 'date'],                                                  
                                                  right_on = ['location_name', 'date'],
                                                how = 'left', 
#                                                  indicator=True 
                                                  )

mask_adherance_merge_outcomes_all_states = mask_adherance_merge_outcomes_all_states.merge(NOAA_precipitation,
                                                  left_on = ['Statename', 'date'],                                                  
                                                  right_on = ['location_name', 'date'],
                                                how = 'left', 
#                                                  indicator=True 
                                                  )

print('NOAA_temperature', NOAA_temperature.shape,
      '\n mask_adherance_merge_outcomes_all_states', mask_adherance_merge_outcomes_all_states.shape)



mask_adherance_merge_outcomes_all_states.head()

In [ ]:
pd.unique(mask_adherance_merge_outcomes_all_states['state'])

In [ ]:
mask_adherance_merge_outcomes_all_states.to_csv('data/output/mask_adherance_merge_outcomes_all_states_produced_{}.csv'.format(
    str(time_produced.strftime("%Y-%m-%d %H.%M"))), index=False)

print('data/output/mask_adherance_merge_outcomes_all_states_produced_{}.csv'.format(
    str(time_produced.strftime("%Y-%m-%d %H.%M"))))

# early vs late wave states

In [ ]:
merged_outcome_df.head()

In [ ]:
column = 'confirmed_cases'
plot_data = merged_outcome_df
fig, ax = plt.subplots(figsize=(20, 10))
for label, grp in plot_data.groupby('state'):
#     if label not in first_wave_states_list:
#         continue
    x = grp['day']
    y = grp[column]
    plt.plot(x, y, label = label, alpha = 0.8)
plt.legend(loc="lower left", ncol=16)

In [ ]:
column = 'confirmed_cases'

first_wave = merged_outcome_df[merged_outcome_df['day'] == pd.to_datetime('2020-04-15')]
first_wave = first_wave.sort_values(by=[column], ascending = False)
first_wave_states_list = first_wave['state'].to_list()[:15]
first_wave_states_list = first_wave_states_list + ['WA']


print(first_wave_states_list)

plot_data = merged_outcome_df
fig, ax = plt.subplots(figsize=(20, 10))
for label, grp in plot_data.groupby('state'):
    if label not in first_wave_states_list:
        continue
    x = grp['day']
    y = grp[column]
    plt.plot(x, y, label = label, alpha = 0.8)
#     break
plt.legend(loc="lower left", ncol=16)
# fig.suptitle('State Revenue Daily', fontsize=20)
# plt.xlabel('Day', fontsize=18)
plt.ylabel(column, fontsize=16)